In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc
import ast
from nn_rag import Knowledge
from tqdm.auto import tqdm
from spacy.lang.en import English

In [3]:
kn = Knowledge.from_memory()

In [18]:
text = kn.__class__.__module__
text.partition('.')[0]


'nn_rag'